In [1]:
# importation of the packages

import numpy as np
from graph_tool.all import *
import random
import graph_tool.topology as gt
import graph_tool.clustering as gc
import graph_tool.centrality as gcent
import graph_tool.generation as gg

import matplotlib.pyplot as plt
import pickle
import matplotlib.colors as colors
import matplotlib
import copy
from matplotlib.lines import Line2D 
from collections import defaultdict

from scipy.stats.stats import pearsonr
import pandas as pd

%matplotlib inline
import seaborn as sns
from scipy.stats import gaussian_kde
from collections import Counter
import math
import scipy.stats
import collections

import matplotlib.ticker as mtick
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import norm 
from sklearn.neighbors import KernelDensity 
from sklearn.utils.fixes import parse_version 

from sklearn.svm import SVC # "Support vector classifier"
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score

from collections import Counter
#from tabulate import tabulate
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC

from multiprocessing import Pool
from  matplotlib.colors import LinearSegmentedColormap

from scipy.stats.stats import pearsonr 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import os

## A-Useful functions

In [2]:
##########################################################################################################

def delta(a,b):
    """
    Return 1 if a and b are identical, else 0
    """
    if a == b:
        return 1
    else:
        return 0
    
##########################################################################################################

def theta(x):
    """
    Return 1 if x is positif, else 0
    """
    if x<0:
        return(0)
    else:
        return(1)

##########################################################################################################

def likelihood_only_SI(t,beta,rate,T_SI_node,T_SI_neighbors):
    """
    Return the likelihood that node i is in state T[i,t] at time t if infected by the simple contagion
    Input:
    - t: the time
    - beta: parameter of the simple contagion
    - rate: rate of infection of the spontaneous adoption
    - T_SI_node: trajectory of the node i
    - T_SI_neighbors: trjectory of the nighbours of node i
    
    """
    prod = np.prod([(1-beta)**(T_SI_N[t-1]) for T_SI_N in T_SI_neighbors]) # it is pi(1-beta)
                                                           
    SI_no_exponent = (1-prod)*(1-rate)
    SI = SI_no_exponent**(T_SI_node[t]*(1-T_SI_node[t-1]))
                                            
    SS_no_exponent = prod * (1-rate)
    SS = SS_no_exponent**((1-T_SI_node[t])*(1-T_SI_node[t-1]))
    
    IS = 0 ** (T_SI_node[t-1]*(1-T_SI_node[t]))
    
    II = 1 ** (T_SI_node[t-1]*T_SI_node[t])
    
    return(SI*SS*IS*II)

##########################################################################################################

def likelihood_random_SI(t,beta,rate,T_SI_node,T_SI_neighbors):
    """
    Return the likelihood that node i is in state T[i,t] at time t if infected by the spontaneous contagion, assigned initially simple contagion
    Input:
    - t: the time
    - beta: parameter of the simple contagion
    - rate: rate of infection of the spontaneous adoption
    - T_SI_node: trajectory of the node i
    - T_SI_neighbors: trjectory of the nighbours of node i
    
    """
    
    prod = np.prod([(1-beta)**(T_SI_N[t-1]) for T_SI_N in T_SI_neighbors]) # it is pi(1-beta)
                                                           
    SI_no_exponent = rate *prod
    SI = SI_no_exponent**(T_SI_node[t]*(1-T_SI_node[t-1]))
                                            
    SS_no_exponent = prod * (1-rate)
    SS = SS_no_exponent**((1-T_SI_node[t])*(1-T_SI_node[t-1]))
    
    IS = 0 ** (T_SI_node[t-1]*(1-T_SI_node[t]))
    
    II = 1 ** (T_SI_node[t-1]*T_SI_node[t])
    
    return(SI*SS*IS*II)

######################################################################################################

def prod_likelihood_only_SI(x):
    
    """
    multiplication over all the time steps of the likelihood that node i is in state T[i,t] at time t if infected by the simple contagion
    Input:
    x: one row of the dataset, countaining the information of infection of a node
    """
    
    i = int(x['index_node'])
    T_SI_node = x['trajectory_node']
    T_SI_neighbors = x['trajectory_neighbors']
    number_time = len(T_SI_node)
    prod = np.prod([likelihood_only_SI(t,beta,rate,T_SI_node,T_SI_neighbors) for t in range(1,number_time)])
    return(prod)   

######################################################################################################

def prod_likelihood_random_SI(x):
    
    """
    multiplication over all the time steps of the likelihood that node i is in state T[i,t] at time t if infected by the spontaneous contagion, assigned initially simple contagion
    Input:
    x: one row of the dataset, countaining the information of infection of a node
    """
    
    i = int(x['index_node'])
    T_SI_node = x['trajectory_node']
    T_SI_neighbors = x['trajectory_neighbors']
    number_time = len(T_SI_node)
    prod = np.prod([likelihood_random_SI(t,beta,rate,T_SI_node,T_SI_neighbors) for t in range(1,number_time)])
    return(prod) 

#######################################################################################################

def likelihood_only_CP(t,phi,rate,T_CP_node,T_CP_neighbors):
    """
    Return the likelihood that node i is in state T[i,t] at time t if infected by the complex contagion
    Input:
    - t: the time
    - phi: parameter of the complex contagion
    - rate: rate of infection of the spontaneous adoption
    - T_SI_node: trajectory of the node i
    - T_SI_neighbors: trjectory of the nighbours of node i
    
    """
    
    nber_neighbors = len(T_CP_neighbors)
    
    if (nber_neighbors != 0):
        prop_infected_neighb = sum([T_CP_N[t-1] for T_CP_N in T_CP_neighbors])/nber_neighbors
    else:
        prop_infected_neighb = 0
        
    condition = theta(prop_infected_neighb - phi)
    
    SI_no_exponent = condition
    SI = SI_no_exponent**(T_CP_node[t]*(1-T_CP_node[t-1]))
                                            
    SS_no_exponent = (1 - condition) * (1-rate)
    SS = SS_no_exponent**((1-T_CP_node[t])*(1-T_CP_node[t-1]))
    
    IS = 0 ** (T_CP_node[t-1]*(1-T_CP_node[t]))
    
    II = 1 ** (T_CP_node[t-1]*T_CP_node[t])
    
    return(SI*SS*IS*II)

def likelihood_random_CP(t,phi,rate,T_CP_node,T_CP_neighbors):
    
    """
    Return the likelihood that node i is in state T[i,t] at time t if infected by the spontaneous contagion, assigned initially complex contagion
    Input:
    - t: the time
    - phi: parameter of the complex contagion
    - rate: rate of infection of the spontaneous adoption
    - T_SI_node: trajectory of the node i
    - T_SI_neighbors: trjectory of the nighbours of node i
    
    """
    
    nber_neighbors = len(T_CP_neighbors)
    
    if (nber_neighbors != 0):
        prop_infected_neighb = sum([T_CP_N[t-1] for T_CP_N in T_CP_neighbors])/nber_neighbors
    else:
        prop_infected_neighb = 0
        
    condition = theta(prop_infected_neighb - phi)
    
    SI_no_exponent = rate * (1-condition)
    SI = SI_no_exponent**(T_CP_node[t]*(1-T_CP_node[t-1]))
                                            
    SS_no_exponent = (1 - condition) * (1-rate)
    SS = SS_no_exponent**((1-T_CP_node[t])*(1-T_CP_node[t-1]))
    
    IS = 0 ** (T_CP_node[t-1]*(1-T_CP_node[t]))
    
    II = 1 ** (T_CP_node[t-1]*T_CP_node[t])
    
    return(SI*SS*IS*II)

############################################################################################################

def prod_likelihood_only_CP(x):
    
    """
    multiplication over all the time steps of the likelihood that node i is in state T[i,t] at time t if infected by the complex contagion
    Input:
    x: one row of the dataset, countaining the information of infection of a node
    """
    
    i = int(x['index_node'])
    T_CP_node = x['trajectory_node']
    T_CP_neighbors = x['trajectory_neighbors']
    number_time = len(T_CP_node)
    prod = np.prod([likelihood_only_CP(t,phi,rate,T_CP_node,T_CP_neighbors) for t in range(1,number_time)])
    return(prod) 

############################################################################################################

def prod_likelihood_random_CP(x):
    
    """
    multiplication over all the time steps of the likelihood that node i is in state T[i,t] at time t if infected by the spontaneous contagion, assigned initially complex contagion
    Input:
    x: one row of the dataset, countaining the information of infection of a node
    """
    
    i = int(x['index_node'])
    T_CP_node = x['trajectory_node']
    T_CP_neighbors = x['trajectory_neighbors']
    number_time = len(T_CP_node)
    prod = np.prod([likelihood_random_CP(t,phi,rate,T_CP_node,T_CP_neighbors) for t in range(1,number_time)])
    return(prod) 

##############################################################################################################

def from_adjmatrix_to_adjlist(A):
    
    """
    Function which takes in input the adjacency matrix and returns the adjacency list
    """
    
    A = A.toarray()
    adjList = defaultdict(list)
    for i in range(np.shape(A)[0]):
        for j in range(np.shape(A[i])[0]):
            if A[i][j]== 1:
                adjList[i].append(j)
    return(adjList)

###############################################################################################################

def ratio(x):
    
    """
    Adding a row in the dataset of the ratio likelihood of being complex / likelihood of being simple
    """
    
    num = x['llh_CP']
    den = x['llh_SI']
    
    if (den != 0):
        return(num/den)
    else:
        return(np.nan)
    
###############################################################################################################

def guess(x):
    
    """
    Returning the inferred classification, 0 for simple, 1 for complex and 2 if it is spontaneous
    
    """
    
    x_only_SI = x.llh_only_SI
    x_random_SI = x.llh_random_SI
    
    x_only_CP = x.llh_only_CP
    x_random_CP = x.llh_random_CP
    
    m = max([x_only_SI, x_random_SI, x_only_CP, x_random_CP])
    
    if (m == x_only_SI) & (m!=x_only_CP) & (m!=x_random_SI) & (m!=x_random_CP):
        return(0)
        
    elif (m == x_only_CP) & (m!=x_only_SI) & (m!=x_random_SI) & (m!=x_random_CP):
        return(1)
        
    elif (m == x_random_SI) & (m!=x_only_SI) & (m!=x_only_CP):
        return(2)
    
    elif (m == x_random_CP) & (m!=x_only_SI) & (m!=x_only_CP):
        return(2)
        
    else:
        return ('not comparable')

# B- getting the path of the data folder

In [3]:
x_dir_data = os.getcwd()[:-8]+'data'

# C- making the classification on the whole dataset of Experiment 2

In [5]:
network = 'ER'
list_beta = [0.1, 0.3, 0.5, 0.7, 0.9]
list_phi = [0.1, 0.3, 0.5, 0.7, 0.9]
sample_size = 2000
rate = 0.005
name_cascade = 'general'

In [ ]:
for network in ['ER', 'BA', 'WS', 'SBM', 'twitter']: # 'BA', 'WS', 'SBM' 'ER', 
    
    print(network)

    for index_beta, beta in enumerate(list_beta):
        for index_phi, phi in enumerate(list_phi):
            for it in range(10):

                #################################################################
                # loading of the data base
                with open(x_dir_data+'/df_experiment2_3/df_'+network+'_premix_different_parameter.pickle', 'rb') as handle:
                    df_general = pickle.load(handle)

                df_SI = df_general[(df_general['contagion'] == 0) & (df_general['parameter'] == beta)]
                df_CP = df_general[(df_general['contagion'] == 1) & (df_general['parameter'] == phi)]

                # random infection written as 2 in the contagion column
                df_SI['contagion'] = np.where(df_SI.is_seed == 2,2, df_SI['contagion'])
                df_CP['contagion'] = np.where(df_CP.is_seed == 2,2, df_CP['contagion'])

                # seeds written as 2 in the contagion column
                df_SI['contagion'] = np.where(df_SI.is_seed == 1,2, df_SI['contagion'])
                df_CP['contagion'] = np.where(df_CP.is_seed == 1,2, df_CP['contagion'])

                df_random = pd.concat([df_SI[df_SI['contagion'] == 2],df_CP[df_CP['contagion'] == 2]])
                df_SI = df_SI[df_SI['contagion'] == 0]
                df_CP = df_CP[df_CP['contagion'] == 1]
                #################################################################

                # bootstap, we select sample_size
                df_sample_SI = df_SI.sample(n=int(sample_size), replace=True, axis=0) 
                df_sample_CP = df_CP.sample(n=int(sample_size), replace=True, axis=0)
                df_sample_random = df_random.sample(n=int(sample_size), replace=True, axis=0)

                # knowing it is SI, likelihood it is SI and CP
                df_sample_SI['llh_only_SI'] = df_sample_SI.apply(prod_likelihood_only_SI, axis=1)
                df_sample_SI['llh_random_SI'] = df_sample_SI.apply(prod_likelihood_random_SI, axis=1)
                df_sample_SI['llh_only_CP'] = df_sample_SI.apply(prod_likelihood_only_CP, axis=1)
                df_sample_SI['llh_random_CP'] = df_sample_SI.apply(prod_likelihood_random_CP, axis=1)

                # knowing it is CP, likelihood it is SI and CP
                df_sample_CP['llh_only_SI'] = df_sample_CP.apply(prod_likelihood_only_SI, axis=1)
                df_sample_CP['llh_random_SI'] = df_sample_CP.apply(prod_likelihood_random_SI, axis=1)
                df_sample_CP['llh_only_CP'] = df_sample_CP.apply(prod_likelihood_only_CP, axis=1)
                df_sample_CP['llh_random_CP'] = df_sample_CP.apply(prod_likelihood_random_CP, axis=1)

                # knowing it is random, likelihood it is SI and CP
                df_sample_random['llh_only_SI'] = df_sample_random.apply(prod_likelihood_only_SI, axis=1)
                df_sample_random['llh_random_SI'] = df_sample_random.apply(prod_likelihood_random_SI, axis=1)
                df_sample_random['llh_only_CP'] = df_sample_random.apply(prod_likelihood_only_CP, axis=1)
                df_sample_random['llh_random_CP'] = df_sample_random.apply(prod_likelihood_random_CP, axis=1)


                df_sample_SI['guess'] = df_sample_SI.apply(guess, axis = 1)
                df_sample_CP['guess'] = df_sample_CP.apply(guess, axis = 1)
                df_sample_random['guess'] = df_sample_random.apply(guess, axis = 1)

                # construction of the confusion matrix
                confusion_matrix = np.zeros((3,3))

                confusion_matrix[0, 0] = df_sample_SI['guess'][df_sample_SI['guess'] == 0].count()
                confusion_matrix[1, 0] = df_sample_SI['guess'][df_sample_SI['guess'] == 1].count()
                confusion_matrix[2, 0] = df_sample_SI['guess'][df_sample_SI['guess'] == 2].count()

                confusion_matrix[0, 1] = df_sample_CP['guess'][df_sample_CP['guess'] == 0].count()
                confusion_matrix[1, 1] = df_sample_CP['guess'][df_sample_CP['guess'] == 1].count()
                confusion_matrix[2, 1] = df_sample_CP['guess'][df_sample_CP['guess'] == 2].count()

                confusion_matrix[0, 2] = df_sample_random['guess'][df_sample_random['guess'] == 0].count()
                confusion_matrix[1, 2] = df_sample_random['guess'][df_sample_random['guess'] == 1].count()
                confusion_matrix[2, 2] = df_sample_random['guess'][df_sample_random['guess'] == 2].count()

                with open('_llh_experiment2_confusion_matrix_'+str(network)+'_'+str(beta*10)+'_phi_'+str(phi*10)+'_it_'+str(it)+'_premix_different_parameter.pickle', 'wb') as handle:
                    pickle.dump(confusion_matrix, handle)

ER
beta 0.1 phi 0.1 it 0


/tmp/ipykernel_41891/2364786487.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SI['contagion'] = np.where(df_SI.is_seed == 2,2, df_SI['contagion'])
/tmp/ipykernel_41891/2364786487.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CP['contagion'] = np.where(df_CP.is_seed == 2,2, df_CP['contagion'])
/tmp/ipykernel_41891/2364786487.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

beta 0.1 phi 0.1 it 1
beta 0.1 phi 0.1 it 2
beta 0.1 phi 0.1 it 3
beta 0.1 phi 0.1 it 4
beta 0.1 phi 0.1 it 5
beta 0.1 phi 0.1 it 6
beta 0.1 phi 0.1 it 7
beta 0.1 phi 0.1 it 8
beta 0.1 phi 0.1 it 9
beta 0.1 phi 0.3 it 0
beta 0.1 phi 0.3 it 1
beta 0.1 phi 0.3 it 2
beta 0.1 phi 0.3 it 3
beta 0.1 phi 0.3 it 4
beta 0.1 phi 0.3 it 5
beta 0.1 phi 0.3 it 6
beta 0.1 phi 0.3 it 7
beta 0.1 phi 0.3 it 8
beta 0.1 phi 0.3 it 9
beta 0.1 phi 0.5 it 0
beta 0.1 phi 0.5 it 1
beta 0.1 phi 0.5 it 2
beta 0.1 phi 0.5 it 3
beta 0.1 phi 0.5 it 4
beta 0.1 phi 0.5 it 5
beta 0.1 phi 0.5 it 6
beta 0.1 phi 0.5 it 7
beta 0.1 phi 0.5 it 8
beta 0.1 phi 0.5 it 9
beta 0.1 phi 0.7 it 0
beta 0.1 phi 0.7 it 1
beta 0.1 phi 0.7 it 2
beta 0.1 phi 0.7 it 3
beta 0.1 phi 0.7 it 4
beta 0.1 phi 0.7 it 5
beta 0.1 phi 0.7 it 6
beta 0.1 phi 0.7 it 7
beta 0.1 phi 0.7 it 8
beta 0.1 phi 0.7 it 9
beta 0.1 phi 0.9 it 0
beta 0.1 phi 0.9 it 1
beta 0.1 phi 0.9 it 2
beta 0.1 phi 0.9 it 3
beta 0.1 phi 0.9 it 4
beta 0.1 phi 0.9 it 5
beta 0.1 p

beta 0.5 phi 0.5 it 5
beta 0.5 phi 0.5 it 6
beta 0.5 phi 0.5 it 7
beta 0.5 phi 0.5 it 8
beta 0.5 phi 0.5 it 9
beta 0.5 phi 0.7 it 0
beta 0.5 phi 0.7 it 1
beta 0.5 phi 0.7 it 2
beta 0.5 phi 0.7 it 3
beta 0.5 phi 0.7 it 4
beta 0.5 phi 0.7 it 5
beta 0.5 phi 0.7 it 6
beta 0.5 phi 0.7 it 7
beta 0.5 phi 0.7 it 8
beta 0.5 phi 0.7 it 9
beta 0.5 phi 0.9 it 0
beta 0.5 phi 0.9 it 1
beta 0.5 phi 0.9 it 2
beta 0.5 phi 0.9 it 3
beta 0.5 phi 0.9 it 4
beta 0.5 phi 0.9 it 5
beta 0.5 phi 0.9 it 6
beta 0.5 phi 0.9 it 7
beta 0.5 phi 0.9 it 8
beta 0.5 phi 0.9 it 9
beta 0.7 phi 0.1 it 0
beta 0.7 phi 0.1 it 1
beta 0.7 phi 0.1 it 2
beta 0.7 phi 0.1 it 3
beta 0.7 phi 0.1 it 4
beta 0.7 phi 0.1 it 5
beta 0.7 phi 0.1 it 6
beta 0.7 phi 0.1 it 7
beta 0.7 phi 0.1 it 8
beta 0.7 phi 0.1 it 9
beta 0.7 phi 0.3 it 0
beta 0.7 phi 0.3 it 1
beta 0.7 phi 0.3 it 2
beta 0.7 phi 0.3 it 3
beta 0.7 phi 0.3 it 4
beta 0.7 phi 0.3 it 5
beta 0.7 phi 0.3 it 6
beta 0.7 phi 0.3 it 7
beta 0.7 phi 0.3 it 8
beta 0.7 phi 0.3 it 9
beta 0.7 p

beta 0.9 phi 0.9 it 8
beta 0.9 phi 0.9 it 9
SBM
beta 0.1 phi 0.1 it 0
beta 0.1 phi 0.1 it 1
beta 0.1 phi 0.1 it 2
beta 0.1 phi 0.1 it 3
beta 0.1 phi 0.1 it 4
beta 0.1 phi 0.1 it 5
beta 0.1 phi 0.1 it 6
beta 0.1 phi 0.1 it 7
beta 0.1 phi 0.1 it 8
beta 0.1 phi 0.1 it 9
beta 0.1 phi 0.3 it 0
beta 0.1 phi 0.3 it 1
beta 0.1 phi 0.3 it 2
beta 0.1 phi 0.3 it 3
beta 0.1 phi 0.3 it 4
beta 0.1 phi 0.3 it 5
beta 0.1 phi 0.3 it 6
beta 0.1 phi 0.3 it 7
beta 0.1 phi 0.3 it 8
beta 0.1 phi 0.3 it 9
beta 0.1 phi 0.5 it 0
beta 0.1 phi 0.5 it 1
beta 0.1 phi 0.5 it 2
beta 0.1 phi 0.5 it 3
beta 0.1 phi 0.5 it 4
beta 0.1 phi 0.5 it 5
beta 0.1 phi 0.5 it 6
beta 0.1 phi 0.5 it 7
beta 0.1 phi 0.5 it 8
beta 0.1 phi 0.5 it 9
beta 0.1 phi 0.7 it 0
beta 0.1 phi 0.7 it 1
beta 0.1 phi 0.7 it 2
beta 0.1 phi 0.7 it 3
beta 0.1 phi 0.7 it 4
beta 0.1 phi 0.7 it 5
beta 0.1 phi 0.7 it 6
beta 0.1 phi 0.7 it 7
beta 0.1 phi 0.7 it 8
beta 0.1 phi 0.7 it 9
beta 0.1 phi 0.9 it 0
beta 0.1 phi 0.9 it 1
beta 0.1 phi 0.9 it 2
beta 0

In [ ]:
# end of the code :)